In [12]:
import numpy as np
import time
# import ldpc
# import bposd
# from bposd.css import css_code

In [9]:
def BinaryRepMat(mat):
    rows = [int(''.join(map(str, list(row))), 2) for row in mat]
#     for row in mat:
#         binary_str = ''.join(map(str, list(row)))
#         rows.append(int(binary_str, 2))
        
    return rows
        
def rankBinaryMat(mat):
    """
    Find rank of a matrix over GF2.

    The rows of the matrix are given as nonnegative integers, thought
    of as bit-strings.

    This function modifies the input list. Use gf2_rank(rows.copy())
    instead of gf2_rank(rows) to avoid modifying rows.
    """
    rows = BinaryRepMat(mat)
    rank = 0
    while rows:
        pivot_row = rows.pop()
        if pivot_row:
            rank += 1
            lsb = pivot_row & -pivot_row
            for index, row in enumerate(rows):
                if row & lsb:
                    rows[index] = row ^ pivot_row
    return rank

# def rankBinaryMat(A):
#     n = len(A[0])
#     rank = 0
#     for col in range(n):
#         j = 0
#         rows = []
#         while j < len(A):
#             if A[j][col] == 1:
#                 rows += [j]
#             j += 1
#         if len(rows) >= 1:
#             for c in range(1,len(rows)):
#                 for k in range(n):
#                     A[rows[c]][k] = (A[rows[c]][k] +A[rows[0]][k])%2
#             A.pop(rows[0])
#             rank += 1
#     for row in A:
#         if sum(row) > 0:
#             rank += 1
#     return rank


def mod_HM(num):
    if np.abs(num - np.round(num)) < 1e-2:
        num -= num - np.round(num)
    return np.mod(num, 1)

def FromPauliToPCMatrix(pauli_operators):
    pauli_stabilizers = pauli_operators
    pauli_logical_xs = (pauli_operators[0])
    pauli_logical_zs = (pauli_operators[0])
    eval_code = BasicCode(pauli_stabilizers=pauli_stabilizers, pauli_logical_xs=pauli_logical_xs, pauli_logical_zs=pauli_logical_zs)
    return eval_code.stabilizers

def IfStabilizer(code_stabilizer_matrix, eval_1chain_vec):
    stabilizer_rank = rankBinaryMat(np.ndarray.tolist(code_stabilizer_matrix))
    joined_matrix = np.vstack([code_stabilizer_matrix, eval_1chain_vec])
    joined_rank = rankBinaryMat(np.ndarray.tolist(joined_matrix))
    if joined_rank > stabilizer_rank:
        return False
    else:
        return True

def SympleticMatrixProduct(mat1, mat2, N):
    zero_matrix = np.zeros([N,N])
    identity_matrix = np.identity(N)
    I0_s = np.matrix(np.vstack([np.hstack([zero_matrix, identity_matrix]), np.hstack([identity_matrix, zero_matrix])]))
    return mat1*I0_s*mat2%2
    
def IfCommute(eval_1chain_vec, eval_2chain_vec):
    #N = int(eval_1chain_vec.shape[1]/2)
    N = int(len(eval_1chain_vec)/2)
    #print(N)
    commu_vec = SympleticMatrixProduct(np.matrix(eval_1chain_vec), np.transpose(np.matrix(eval_2chain_vec)), N)
    if np.sum(commu_vec) > 0:
        return False
    else:
        return True
    
def IfCenterlizer(code_stabilizer_matrix, eval_1chain_vec):
    N = int(code_stabilizer_matrix.shape[1]/2)
    commu_vec = SympleticMatrixProduct(np.matrix(code_stabilizer_matrix), np.transpose(np.matrix(eval_1chain_vec)), N)
    if np.sum(commu_vec) > 0:
        return False
    else:
        return True
    
def IfLogicalOper(code_stabilizer_matrix, eval_1chain_vec):
    return IfCenterlizer(code_stabilizer_matrix, eval_1chain_vec) & (not IfStabilizer(code_stabilizer_matrix, eval_1chain_vec))


def FromErrorChainToPauli(error_chain):
    n = round(len(error_chain)/2)
    pauli_operator = ''
    for i in range(n):
        if (error_chain[i] == 1) & (error_chain[n + i] == 1):
            pauli_operator += 'Y'
        elif (error_chain[i] == 1):
            pauli_operator += 'X'
        elif (error_chain[n + i] == 1):
            pauli_operator += 'Z'
        else:
            pauli_operator += 'I'
    return pauli_operator

## Load Ramunujan Cayley Complex Codes

#### Load Data now




In [2]:
# Loading the codes
import numpy as np
import pandas as pd 

# Loading the parameters: 
ma = 6
mb = 9 
degree = 14 
basedim = 120 
edgedim = degree * basedim
facedim = degree * edgedim



idTGEF = pd.read_csv('data/p13_q5/ramunujancsscodes_13_5_pgl_TGEF.csv', header=None).values.tolist()
idTGVE = pd.read_csv('data/p13_q5/ramunujancsscodes_13_5_pgl_TGVE.csv', header=None).values.tolist()
print(len(idTGEF))
print(len(idTGVE))

168000
86400


In [4]:
from build_caylaycomplexcodes import BuildqcssCodes

qCSS_codes = BuildqcssCodes(ma=6, mb=9, delta=14, basedim=120)
# .build_codes(TGVEhor_idx, TGVEver_idx, TGEFhor_idx, TGEFver_idx)
X, Z = qCSS_codes.build_chains(idTGVE, idTGEF)
print(X.shape)
print(Z.shape)

(6480, 25200)
(23520, 25200)


In [6]:
from scipy import sparse
# Check properties 
# (1) Check for the CSS codes condition HxHz^T = 0

## checking if the CSS condition is sastfied
qCSS_codes.check_exactseq(sparse.csr_matrix(Z.transpose()), sparse.csr_matrix(X))

[]


In [8]:
# (2) check the low-density LDPC codes (apated from Lemma 3.2 from Vidick et al 2022)
qCSS_codes.check_lowdensity(Z, X)
# 3 Check the code rates adapted from the Lemma 3.3 from Vidick et al 2022 and thm 2.10.

# def coderate(X, Z):
#     X2 = np.shape(Z)[0]
#     X1 = 2 * np.shape(Z)[1]
#     X0 = 4 * np.shape(X)[0]
#     return (X1 - X2 - X0) / X1

# print(f'the code rate is at least {coderate(X, Z)}') 

the maximum weight given Z stabilizer: 10, should be bounded by 36 by Cayley Complexes 
the maximum number of Z stabilizers on a given qubit: 10, should be bounded by 14
the maximum weight given X stabilizer: 19, should be bounded by 28
the maximum number of X stabilizers on a given qubit: 6, should be bounded by 18


# Method 1: using the ldpc package

In [ ]:
hx = XChecks['13_5_pgl']
hz = ZChecks['13_5_pgl']
tanner_ff_code1 = css_code(hx=hx,hz=hz)

print('print code parametersP:', tanner_ff_code1.code_params) # format: (\Delta_v, \Delta_c)-"[[n,k,d]]"

# Method 2: compute the rank of H_x + rank of H_Z, then k = n - (rank(H_x) + rank(H_z))

In [13]:
start_time = time.time()
rank_hx = rankBinaryMat(X)
rank_hz = rankBinaryMat(Z)
n = X.shape[1]
k = n - (rank_hx + rank_hz)
print('rank of hx:', rank_hx, 'rank of hz:', rank_hz, 'k:', k)
end_time = time.time()
print('time elapsed:', end_time - start_time)

rank of hx: 6480 rank of hz: 17760 k: 960
time elapsed: 379.5679109096527


In [14]:
print(f'the code rate is {k/n}')

the code rate is 0.0380952380952381
